In [1]:
import gurobipy as gp
from itertools import product
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
def division(a,b):
    if a == 0:
        return 0
    if b == 0:
        return 0
    return a/b

## 1 Modèle de base des centrales thermiques

### 1.1 Planification journalière du parc thermique

Variables de décision :
Pour une type de centrale $X \in [A,B,C]$, et une heure de la journée $t\in {1,\dots,24}$, on définit :
$$
N_t^{(X)} = \text{Nombre de centrales } X \text{allumées à } t\text{ h, (ENTIER)}
$$
$$
P_t^{(X)} = \text{Puissance totale produite par les centrales } X \text{ à } t\text{ h, (CONTINUE)}
$$
Contraintes :
$$
N_t^{(X)} P_{min}^{(X)} \leq P_t^{(X)} \leq N_t^{(X)} P_{max}^{(X)} \text{ ,  Contraintes sur la puissance totale de chaque centrale}
$$
$$
0 \leq N_t^{(X)} \leq N^{(X)} \text{   ,   Contraintes sur le nombre de centrales allumées possible}
$$
$$
\forall t, \sum_X P_t^{(X)} = D_t \text{  ,  Contrainte équilibre offre-demande}
$$
Objectif :
$$
\text{Minimiser} \sum_X \sum_T P_t^{(X)} C_{MWh}^{(X)}
$$

In [2]:
class Centrale:
    def __init__(self, name, N, Pmin, Pmax, Cmwh):
        self.name = name
        self.N = N
        self.Pmin = Pmin
        self.Pmax = Pmax
        self.Cmwh = Cmwh

In [3]:
dict_11 = {}
dict_11["A"] = Centrale(
    name="A",
    N=12,
    Pmax=2000,
    Pmin=850,
    Cmwh=1.5
)
dict_11["B"] = Centrale(
    name="B",
    N=10,
    Pmax=1750,
    Pmin=1250,
    Cmwh=1.38
)
dict_11["C"] = Centrale(
    name="C",
    N=5,
    Pmax=4000,
    Pmin=1500,
    Cmwh=2.75
)
consommation = np.array([15,15,15,15,15,15,30,30,30,25,25,25,25,25,25,40,40,40,27,27,27,27,27,27])*1000

In [4]:
model = gp.Model(name="1.1")

Set parameter TokenServer to value "dev.cma.mines-paristech.fr"


In [5]:
dict_N = {}
dict_P = {}

for t in range(24):
    for X in dict_11:
        dict_N[X,t] = model.addVar(lb=0,ub=dict_11[X].N,vtype=gp.GRB.INTEGER,name=f"Nombre de centrale {X} allumées à {t}h")
    for X in dict_11:
        dict_P[X,t] = model.addVar(name=f"Puissance totale {X} à {t}h")


In [6]:

model.setObjective(gp.quicksum([dict_P[element[0],element[1]] * dict_11[element[0]].Cmwh for element in product(dict_11,range(24))]), gp.GRB.MINIMIZE)

In [7]:
for t in range(24):
    for X in dict_11:
        model.addConstr(dict_P[X,t]<=dict_N[X,t] * dict_11[X].Pmax, name=f"borne sup puissance, {X} à {t}h")
        model.addConstr(dict_P[X,t]>=dict_N[X,t] * dict_11[X].Pmin, name=f"borne inf puissance, {X} à {t}h")
    model.addConstr(gp.quicksum([dict_P[X,t] for X in dict_11])==consommation[t])

In [8]:
model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 168 rows, 144 columns and 360 nonzeros
Model fingerprint: 0xe0a5527a
Variable types: 72 continuous, 72 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e+00, 3e+00]
  Bounds range     [5e+00, 1e+01]
  RHS range        [2e+04, 4e+04]
Found heuristic solution: objective 1235375.0000
Presolve removed 162 rows and 139 columns
Presolve time: 0.02s
Presolved: 6 rows, 5 columns, 14 nonzeros
Found heuristic solution: objective 881275.00000
Variable types: 2 continuous, 3 integer (0 binary)

Root relaxation: objective 8.694000e+05, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 869400.000    0    1 881275.000 869400.000  1.35%     -    

In [9]:
print(f"Coût : {model.ObjVal}")

Coût : 869400.0


In [10]:
def df_results(dict_N,dict_P,dict_XX):
    df = pd.DataFrame()
    df["h"] = range(24) 
    df["Consommation (MW)"] = consommation
    df["Production total"] = 0
    df["Coût total"] = 0
    for X in dict_XX:
        df[f"Nb centrale {X}"] = [int(dict_N[X,t].X) for t in range(24)]
        df[f"Puissance tot {X}"] = [dict_P[X,t].X for t in range(24)]
        df[f"Coût {X}"] = [dict_P[X,t].X * dict_XX[X].Cmwh for t in range(24)]
        df[f"Facteur de charge{X}"] = [int(division(dict_P[X,t].X ,dict_N[X,t].X * dict_11[X].Pmax)*100) for t in range(24)]
        df["Production total"] += df[f"Puissance tot {X}"]
        df["Coût total"] += df[f"Coût {X}"]
    df["Coût MWh"] = df["Coût total"]/df["Production total"]
    return df
df = df_results(dict_N,dict_P,dict_11)
df

,h,Consommation (MW),Production total,Coût total,Nb centrale A,Puissance tot A,Coût A,Facteur de chargeA,Nb centrale B,Puissance tot B,Coût B,Facteur de chargeB,Nb centrale C,Puissance tot C,Coût C,Facteur de chargeC,Coût MWh
0,0,15000,15000.0,20700.0,0,0.0,0.0,0,9,15000.0,20700.0,95,0,0.0,0.0,0,1.380000
1,1,15000,15000.0,20700.0,0,0.0,0.0,0,9,15000.0,20700.0,95,0,0.0,0.0,0,1.380000
2,2,15000,15000.0,20700.0,0,0.0,0.0,0,9,15000.0,20700.0,95,0,0.0,0.0,0,1.380000
3,3,15000,15000.0,20700.0,0,0.0,0.0,0,9,15000.0,20700.0,95,0,0.0,0.0,0,1.380000
4,4,15000,15000.0,20700.0,0,0.0,0.0,0,9,15000.0,20700.0,95,0,0.0,0.0,0,1.380000
5,5,15000,15000.0,20700.0,0,0.0,0.0,0,9,15000.0,20700.0,95,0,0.0,0.0,0,1.380000
6,6,30000,30000.0,42900.0,12,12500.0,18750.0,52,10,17500.0,24150.0,100,0,0.0,0.0,0,1.430000
7,7,30000,30000.0,42900.0,12,12500.0,18750.0,52,10,17500.0,24150.0,100,0,0.0,0.0,0,1.430000
8,8,30000,30000.0,42900.0,12,12500.0,18750.0,52,10,17500.0,24150.0,100,0,0.0,0.0,0,1.430000
9,9,25000,25000.0,35400.0,4,7500.0,11250.0,93,10,17500.0,24150.0,100,0,0.0,0.0,0,1.416000


In [11]:
from turtle import color
import plotly.graph_objects as go

def Results_viz(df):
    fig = go.Figure()

    Y = df["Consommation (MW)"].copy()*0
    for X in dict_11:
        Y += df[f"Puissance tot {X}"]
        fig.add_trace(
            go.Scatter(
            x=df["h"], 
            y=Y, 
            fill='tonexty',
            name = f"Production {X}"
            )
        )

    # fig.add_trace(
    #     go.Scatter(
    #         x = df["h"],
    #         y = df["Consommation (MW)"],
    #         name = "Consommation"
    #     )
    # )
    fig.add_trace(
        go.Scatter(
            x = df["h"],
            y = df["Production total"],
            name="Production total",
            line = dict(dash='dash',color="red")
        )
    )
    fig.add_trace(
        go.Scatter(
            x=df["h"],
            y=df["Coût MWh"],
            yaxis="y2",
            name="Coût MWh",
            line_color="black"
        )
    )

    fig.update_layout(
        hovermode='x',
        yaxis=dict(title="MW" ,range=[0,df["Consommation (MW)"].max()*1.1]),
        yaxis2=dict(title="€/MWh",
        range=[df["Coût MWh"].min()*0.95,df["Coût MWh"].max()*1.1],
        anchor="free",
        overlaying="y",
        side="right",
        position=1
        ),
        title = "Répartition de la production électrique dans la journée"
    )
    return fig
fig = Results_viz(df)
fig.show()

In [12]:
model_relax = model.relax()
model_relax.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 168 rows, 144 columns and 360 nonzeros
Model fingerprint: 0x48de0531
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e+00, 3e+00]
  Bounds range     [5e+00, 1e+01]
  RHS range        [2e+04, 4e+04]
Presolve removed 153 rows and 99 columns
Presolve time: 0.01s
Presolved: 15 rows, 45 columns, 45 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.5266000e+05   1.743750e+04   0.000000e+00      0s
      15    8.6940000e+05   0.000000e+00   0.000000e+00      0s

Solved in 15 iterations and 0.01 seconds (0.00 work units)
Optimal objective  8.694000000e+05


In [13]:
print(f"Coût : {model_relax.ObjVal}")

Coût : 869400.0


## 2 Coût d'opération

### 2.1 Coût de fonctionnement

Pour ce problème on modifie la fonction objectif.<br>
Objectif :
$$
\text{Minimiser} \sum_X \sum_T (P_t^{(X)} - P_{min}^{(X)}N_t^{(X)}) C_{MWh}^{(X)} + N_t^{(X)}C_{base}^{(X)}
$$

In [14]:
class Centrale2:
    def __init__(self, name, N, Pmin, Pmax, Cstart, Cbase, Cmwh):
        self.name = name
        self.N = N
        self.Pmin = Pmin
        self.Pmax = Pmax
        self.Cstart = Cstart
        self.Cbase = Cbase
        self.Cmwh = Cmwh

In [15]:
dict_2 = {}
dict_2["A"] = Centrale2(
    name="A",
    N=12,
    Pmax=2000,
    Pmin=850,
    Cmwh=2,
    Cstart=2000,
    Cbase=1000
)
dict_2["B"] = Centrale2(
    name="B",
    N=10,
    Pmax=1750,
    Pmin=1250,
    Cmwh=1.3,
    Cstart=1000,
    Cbase=2600
)
dict_2["C"] = Centrale2(
    name="C",
    N=5,
    Pmax=4000,
    Pmin=1500,
    Cmwh=3,
    Cstart=500,
    Cbase=3000
)

In [16]:
model = gp.Model(name="2.1")

dict_N = {}
dict_P = {}

for t in range(24):
    for X in dict_2:
        dict_N[X,t] = model.addVar(lb=0,ub=dict_2[X].N,vtype=gp.GRB.INTEGER,name=f"Nombre de centrale {X} allumées à {t}h")
    for X in dict_2:
        dict_P[X,t] = model.addVar(name=f"Puissance totale {X} à {t}h",vtype=gp.GRB.CONTINUOUS)

model.setObjective(gp.quicksum([dict_N[element[0],element[1]] * (dict_2[element[0]].Cbase - dict_2[element[0]].Pmin*dict_2[element[0]].Cmwh) + dict_P[element[0],element[1]] * dict_2[element[0]].Cmwh for element in product(dict_2,range(24))]), gp.GRB.MINIMIZE)

for t in range(24):
    for X in dict_11:
        model.addConstr(dict_P[X,t]<=dict_N[X,t] * dict_2[X].Pmax, name=f"borne sup puissance, {X} à {t}h")
        model.addConstr(dict_P[X,t]>=dict_N[X,t] * dict_2[X].Pmin, name=f"borne inf puissance, {X} à {t}h")
    model.addConstr(gp.quicksum([dict_P[X,t] for X in dict_2])==consommation[t])
    # model.addConstr(gp.quicksum([dict_P[X,t] for X in dict_11])==consommation[t])
model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 168 rows, 144 columns and 360 nonzeros
Model fingerprint: 0x54e335bf
Variable types: 72 continuous, 72 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e+00, 2e+03]
  Bounds range     [5e+00, 1e+01]
  RHS range        [2e+04, 4e+04]
Found heuristic solution: objective 1271875.0000
Presolve removed 162 rows and 139 columns
Presolve time: 0.02s
Presolved: 6 rows, 5 columns, 14 nonzeros
Found heuristic solution: objective 987500.00000
Variable types: 2 continuous, 3 integer (0 binary)
Found heuristic solution: objective 985950.00000

Root relaxation: objective 9.787500e+05, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 978750.000

In [17]:
print(f"Coût : {model.ObjVal}")

Coût : 978900.0


In [18]:
def df_results2(dict_N,dict_P,dict_XX):
    df = pd.DataFrame()
    df["h"] = range(24) 
    df["Consommation (MW)"] = consommation
    df["Production total"] = 0
    df["Coût total"] = 0
    for X in dict_XX:
        df[f"Nb centrale {X}"] = [int(dict_N[X,t].X) for t in range(24)]
        df[f"Puissance tot {X}"] = [dict_P[X,t].X for t in range(24)]
        df[f"Coût {X}"] = [dict_N[X,t].X * dict_XX[X].Cbase + (dict_P[X,t].X - dict_N[X,t].X * dict_XX[X].Pmin) * dict_XX[X].Cmwh for t in range(24)]
        df["Production total"] += df[f"Puissance tot {X}"]
        df["Coût total"] += df[f"Coût {X}"]
    df["Coût MWh"] = df["Coût total"]/df["Production total"]
    return df
df = df_results2(dict_N=dict_N,dict_P=dict_P,dict_XX=dict_2)
fig = Results_viz(df)
fig.show()

### 2.2 Coût du démarrage

Pour ce problème on modifie les variables de décisions et on y adapte les contraintes.

Variables de décision :
Pour une type de centrale $X \in [A,B,C]$, et une heure de la journée $t\in {1,\dots,24}$, on définit :
$$
N_t^{(X)} = \text{Nombre de centrales } X \text{allumées à } t\text{ h, (ENTIER)}
$$
$$
N_{start,t}^{(X)} = \text{Nombre de centrales } X \text{démarrées à } t\text{ h, (ENTIER)}
$$
$$
P_t^{(X)} = \text{Puissance totale produite par les centrales } X \text{ à } t\text{ h, (CONTINUE)}
$$
Contraintes (par convention $N_{-1}^{(X)}=0$):
$$
N_t^{(X)} P_{min}^{(X)} \leq P_t^{(X)} \leq N_t^{(X)} P_{max}^{(X)} \text{ ,  Contraintes sur la puissance totale de chaque centrale}
$$
$$
N_{start,t}^{(X)} \leq N_t^{(X)} \leq N_{start,t}^{(X)}+N_{t-1}^{(X)} \text{   ,   Contraintes sur le nombre de centrales allumées possible}
$$
$$
0 \leq N_{start,t}^{(X)} \leq N^{(X)}-N_{t-1}^{(X)} \text{   ,   Contraintes sur le nombre de centrales démarrable possible}
$$
$$
\sum_X P_t^{(X)} = D_t \text{  ,  Contrainte équilibre offre-demande}
$$
Objectif :
$$
\text{Minimiser} \sum_X \sum_T (P_t^{(X)} - P_{min}^{(X)}N_t^{(X)}) C_{MWh}^{(X)} + N_t^{(X)}C_{base}^{(X)} + N_{start,t}^{(X)}C_{start}^{(X)}
$$

In [19]:
model = gp.Model(name="2.2")

dict_N = {}
dict_P = {}
dict_Nstart = {}

for t in range(24):
    for X in dict_2:
        dict_N[X,t] = model.addVar(lb=0,ub=dict_2[X].N,vtype=gp.GRB.INTEGER,name=f"Nombre de centrale {X} allumées à {t}h")
        dict_Nstart[X,t] = model.addVar(lb=0,ub=dict_2[X].N,vtype=gp.GRB.INTEGER,name=f"Nombre de centrale {X} démarrées à {t}h")
    for X in dict_2:
        dict_P[X,t] = model.addVar(name=f"Puissance totale {X} à {t}h",vtype=gp.GRB.CONTINUOUS)

model.setObjective(gp.quicksum([dict_N[element[0],element[1]] * (dict_2[element[0]].Cbase - dict_2[element[0]].Pmin*dict_2[element[0]].Cmwh) + dict_P[element[0],element[1]] * dict_2[element[0]].Cmwh + dict_Nstart[element[0],element[1]]*dict_2[element[0]].Cstart for element in product(dict_2,range(24))]), gp.GRB.MINIMIZE)

for t in range(24):
    for X in dict_11:
        model.addConstr(dict_P[X,t]<=dict_N[X,t] * dict_2[X].Pmax, name=f"borne sup puissance, {X} à {t}h")
        model.addConstr(dict_P[X,t]>=dict_N[X,t] * dict_2[X].Pmin, name=f"borne inf puissance, {X} à {t}h")
        model.addConstr(dict_Nstart[X,t]<=dict_N[X,t], name = f"Le nombre de centrale {X} en fonctionnement supérieur au nombre démarré à {t}")
        if t ==0:
            model.addConstr(dict_Nstart[X,t]<= dict_2[X].N, name = f"Nombre max de centrale {X} démarable à {t}")
            model.addConstr(dict_N[X,t] <= dict_Nstart[X,t], name = f"Nombre max de centrale {X} en fonctionnement à {t}")
        else:
            model.addConstr(dict_Nstart[X,t]<= dict_2[X].N - dict_N[X,t-1], name = f"Nombre max de centrale {X} démarable à {t}")
            model.addConstr(dict_N[X,t] <= dict_Nstart[X,t] + dict_N[X,t-1], name = f"Nombre max de centrale {X} en fonctionnement à {t}")
    model.addConstr(gp.quicksum([dict_P[X,t] for X in dict_2])==consommation[t])
    # model.addConstr(gp.quicksum([dict_P[X,t] for X in dict_11])==consommation[t])
model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 384 rows, 216 columns and 858 nonzeros
Model fingerprint: 0x11b2ce6a
Variable types: 72 continuous, 144 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e+00, 2e+03]
  Bounds range     [5e+00, 1e+01]
  RHS range        [5e+00, 4e+04]
Found heuristic solution: objective 1474375.0000
Presolve removed 180 rows and 30 columns
Presolve time: 0.00s
Presolved: 204 rows, 186 columns, 522 nonzeros
Variable types: 42 continuous, 144 integer (0 binary)
Found heuristic solution: objective 1379630.0000

Root relaxation: objective 1.011257e+06, 58 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1011257.14    0   26 1379630.00 1011257.14  26.7% 

In [20]:
def df_results22(dict_N,dict_P,dict_Nstart,dict_XX):
    df = pd.DataFrame()
    df["h"] = range(24) 
    df["Consommation (MW)"] = consommation
    df["Production total"] = 0
    df["Coût total"] = 0
    for X in dict_XX:
        df[f"Nb centrale {X}"] = [int(dict_N[X,t].X) for t in range(24)]
        df[f"Puissance tot {X}"] = [dict_P[X,t].X for t in range(24)]
        df[f"Coût {X}"] = [dict_N[X,t].X * dict_XX[X].Cbase + (dict_P[X,t].X - dict_N[X,t].X * dict_XX[X].Pmin) * dict_XX[X].Cmwh + dict_Nstart[X,t].X*dict_2[X].Cstart for t in range(24)]
        df["Production total"] += df[f"Puissance tot {X}"]
        df["Coût total"] += df[f"Coût {X}"]
    df["Coût MWh"] = df["Coût total"]/df["Production total"]
    return df
df = df_results22(dict_N=dict_N,dict_P=dict_P,dict_Nstart=dict_Nstart,dict_XX=dict_2)
fig = Results_viz(df)
fig.show()

## 3 Réserve de puissance

Pour intégrer la réserve de puissance on ajoute la contrainte :
$$
\sum_X N_t^{(X)}P_{max}^{(X)}\geq D_t\times 1,15
$$

In [21]:
for t in range(24):
    model.addConstr(gp.quicksum([dict_N[X,t]*dict_2[X].Pmax for X in dict_2])>= 1.15 * consommation[t], name = f"Réserve de puissance à {t}")
model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 408 rows, 216 columns and 930 nonzeros
Model fingerprint: 0xff8f72cd
Variable types: 72 continuous, 144 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e+00, 2e+03]
  Bounds range     [5e+00, 1e+01]
  RHS range        [5e+00, 5e+04]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint Réserve_de_puissance_à_15 by 4500.000000000

Found heuristic solution: objective 1319775.0000
Presolve removed 180 rows and 30 columns
Presolve time: 0.00s
Presolved: 228 rows, 186 columns, 594 nonzeros
Found heuristic solution: objective 1239990.0000
Variable types: 42 continuous, 144 integer (0 binary)
Found heuristic solution: objective 1214220.0000

Root relaxation: objective 1.012257e+06, 67 iterations, 0.00 seconds (0.00 work units)

In [22]:
df = df_results22(dict_N=dict_N,dict_P=dict_P,dict_Nstart=dict_Nstart,dict_XX=dict_2)
fig = Results_viz(df)
fig.show()

## 4 Planification cyclique

Pour ce problème on modifie la convention $N_{-1}^{(X)}=0$ par $N_{-1}^{(X)}=N_{23}^{(X)}$

In [23]:
model = gp.Model(name="4")

dict_N = {}
dict_P = {}
dict_Nstart = {}

for t in range(24):
    for X in dict_2:
        dict_N[X,t] = model.addVar(lb=0,ub=dict_2[X].N,vtype=gp.GRB.INTEGER,name=f"Nombre de centrale {X} allumées à {t}h")
        dict_Nstart[X,t] = model.addVar(lb=0,ub=dict_2[X].N,vtype=gp.GRB.INTEGER,name=f"Nombre de centrale {X} démarrées à {t}h")
    for X in dict_2:
        dict_P[X,t] = model.addVar(name=f"Puissance totale {X} à {t}h",vtype=gp.GRB.CONTINUOUS)

model.setObjective(gp.quicksum([dict_N[element[0],element[1]] * (dict_2[element[0]].Cbase - dict_2[element[0]].Pmin*dict_2[element[0]].Cmwh) + dict_P[element[0],element[1]] * dict_2[element[0]].Cmwh + dict_Nstart[element[0],element[1]]*dict_2[element[0]].Cstart for element in product(dict_2,range(24))]), gp.GRB.MINIMIZE)

for t in range(24):
    for X in dict_11:
        model.addConstr(dict_P[X,t]<=dict_N[X,t] * dict_2[X].Pmax, name=f"borne sup puissance, {X} à {t}h")
        model.addConstr(dict_P[X,t]>=dict_N[X,t] * dict_2[X].Pmin, name=f"borne inf puissance, {X} à {t}h")
        model.addConstr(dict_Nstart[X,t]<=dict_N[X,t], name = f"Le nombre de centrale {X} en fonctionnement supérieur au nombre démarré à {t}")
        if t ==0:
            model.addConstr(dict_Nstart[X,t]<= dict_2[X].N - dict_N[X,23], name = f"Nombre max de centrale {X} démarable à {t}")
            model.addConstr(dict_N[X,t] <= dict_Nstart[X,t] + dict_N[X,23], name = f"Nombre max de centrale {X} en fonctionnement à {t}")
        else:
            model.addConstr(dict_Nstart[X,t]<= dict_2[X].N - dict_N[X,t-1], name = f"Nombre max de centrale {X} démarable à {t}")
            model.addConstr(dict_N[X,t] <= dict_Nstart[X,t] + dict_N[X,t-1], name = f"Nombre max de centrale {X} en fonctionnement à {t}")
    model.addConstr(gp.quicksum([dict_P[X,t] for X in dict_2])==consommation[t])
    model.addConstr(gp.quicksum([dict_N[X,t]*dict_2[X].Pmax for X in dict_2])>= 1.15 * consommation[t], name = f"Réserve de puissance à {t}")
    # model.addConstr(gp.quicksum([dict_P[X,t] for X in dict_11])==consommation[t])
model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 408 rows, 216 columns and 936 nonzeros
Model fingerprint: 0xbaa4c106
Variable types: 72 continuous, 144 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e+00, 2e+03]
  Bounds range     [5e+00, 1e+01]
  RHS range        [5e+00, 5e+04]
Found heuristic solution: objective 1354515.0000
Presolve removed 177 rows and 27 columns
Presolve time: 0.00s
Presolved: 231 rows, 189 columns, 603 nonzeros
Found heuristic solution: objective 1265490.0000
Variable types: 42 continuous, 147 integer (0 binary)
Found heuristic solution: objective 1227725.0000

Root relaxation: objective 9.855143e+05, 54 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 9

In [31]:
df = df_results22(dict_N=dict_N,dict_P=dict_P,dict_Nstart=dict_Nstart,dict_XX=dict_2)
fig = Results_viz(df)
fig.update_layout(
    title = "Répartition de la production électrique dans la journée - 4"
)
fig.show()
fig.write_html("Répartition de la production électrique dans la journée - 4.html")


## 5 Centrales hydroélectriques

### 5.1

On ajoute les variables de décisions suivantes:<br>
pour $Y\in[9,14]$ (pour les centrales de 900 MW et 1400MW),<br>
$$
H_t^{(Y)} \in \{0,1\} \text{  , vaut 1 si la centrale $Y$ fonctionne à } t \text{ h 0 sinon}
$$
$$
H_{start,t}^{(Y)} \in \{0,1\} \text{  , vaut 1  si la centrale $Y$ démarre à } t \text{ h 0 sinon}
$$
Avec les contraintes :
$$
H_{start,t}^{(Y)} \leq 1 - H_{t-1}^{(Y)} \text{  ,  S'il y a un démarrage alors la centrale n'était pas allumée}
$$
Autre option :
$$
H_{t}^{(Y)} \leq H_{start,t}^{(Y)}  + H_{t-1}^{(Y)} \text{  ,  Si la centrale fonctionne alors elle était allumée ou elle démarre}
$$
$$
H_{start,t}^{(Y)} \leq H_{t}^{(Y)} \text{  ,  La centrale fonctionne lorsqu'elle est est démarrée}
$$
$$
\sum_X P_t^{(X)} + \sum_Y N_t^{(Y)}P^{(Y)}= D_t \text{  ,  Contrainte équilibre offre-demande}
$$
$$
\sum_X N_t^{(X)}P_{max}^{(X)} + \sum_Y P^{(Y)}\geq D_t\times 1,15 \text{  ,  marges de sécurité}
$$
L'objectif mis à jour devient:
$$
\text{Minimiser} \sum_X \sum_t (P_t^{(X)} - P_{min}^{(X)}N_t^{(X)}) C_{MWh}^{(X)} + N_t^{(X)}C_{base}^{(X)} + N_{start,t}^{(X)}C_{start}^{(X)} + \sum_Y \sum_t H_t^{(Y)}C_{base}^{(Y)} + H_t^{(Y)}C_{start}^{(Y)}
$$

In [25]:
class Centrale_hydro:
    def __init__(self, name, P, Cstart, Cheure, debit):
        self.name = name
        self.P = P
        self.Cstart = Cstart
        self.Cheure = Cheure
        self.debit = debit

In [26]:
dict_Hydro = {}
dict_Hydro[9] = Centrale_hydro(
    name = "Hydro 900 MW",
    P = 900,
    Cstart = 1500,
    Cheure=90,
    debit=0.31
)
dict_Hydro[14] = Centrale_hydro(
    name = "Hydro 1400 MW",
    P = 1400,
    Cstart = 1200,
    Cheure=150,
    debit=0.47
)
dict_Thermique = dict_2.copy()

In [36]:
model = gp.Model(name="5")

dict_H = {}
dict_Hstart = {}

for Y in dict_Hydro:
    for t in range(24):
        dict_H[Y,t] = model.addVar(vtype=gp.GRB.BINARY, name = f"Centrale Hydro {Y} fonctionne à l'heure {t}")
        dict_Hstart[Y,t] = model.addVar(vtype=gp.GRB.BINARY, name = f"Centrale Hydro {Y} démarre à l'heure {t}")

dict_N = {}
dict_P = {}
dict_Nstart = {}

for t in range(24):
    for X in dict_Thermique:
        dict_N[X,t] = model.addVar(lb=0,ub=dict_Thermique[X].N,vtype=gp.GRB.INTEGER,name=f"Nombre de centrale {X} allumées à {t}h")
        dict_Nstart[X,t] = model.addVar(lb=0,ub=dict_Thermique[X].N,vtype=gp.GRB.INTEGER,name=f"Nombre de centrale {X} démarrées à {t}h")
    for X in dict_Thermique:
        dict_P[X,t] = model.addVar(name=f"Puissance totale {X} à {t}h",vtype=gp.GRB.CONTINUOUS)

model.setObjective(
    gp.quicksum([dict_N[element[0],element[1]] * (dict_Thermique[element[0]].Cbase - dict_Thermique[element[0]].Pmin*dict_Thermique[element[0]].Cmwh) for element in product(dict_Thermique,range(24))]) + 
    gp.quicksum([dict_P[element[0],element[1]] * dict_Thermique[element[0]].Cmwh for element in product(dict_Thermique,range(24))])+ 
    gp.quicksum([dict_Nstart[element[0],element[1]]*dict_Thermique[element[0]].Cstart for element in product(dict_Thermique,range(24))])+
    gp.quicksum([dict_H[element[0],element[1]] * dict_Hydro[element[0]].Cheure for element in product(dict_Hydro,range(24))])+
    gp.quicksum([dict_Hstart[element[0],element[1]] * dict_Hydro[element[0]].Cstart for element in product(dict_Hydro,range(24))]),
    gp.GRB.MINIMIZE
    )

for t in range(24):
    for X in dict_Thermique:
        model.addConstr(dict_P[X,t]<=dict_N[X,t] * dict_Thermique[X].Pmax, name=f"borne sup puissance, {X} à {t}h")
        model.addConstr(dict_P[X,t]>=dict_N[X,t] * dict_Thermique[X].Pmin, name=f"borne inf puissance, {X} à {t}h")
        model.addConstr(dict_Nstart[X,t]<=dict_N[X,t], name = f"Le nombre de centrale {X} en fonctionnement supérieur au nombre démarré à {t}")
        if t ==0:
            model.addConstr(dict_Nstart[X,t]<= dict_Thermique[X].N - dict_N[X,23], name = f"Nombre max de centrale {X} démarrable à {t}")
            model.addConstr(dict_N[X,t] <= dict_Nstart[X,t] + dict_N[X,23], name = f"Nombre max de centrale {X} en fonctionnement à {t}")
        else:
            model.addConstr(dict_Nstart[X,t]<= dict_Thermique[X].N - dict_N[X,t-1], name = f"Nombre max de centrale {X} démarrable à {t}")
            model.addConstr(dict_N[X,t] <= dict_Nstart[X,t] + dict_N[X,t-1], name = f"Nombre max de centrale {X} en fonctionnement à {t}")
    for Y in dict_Hydro:
        model.addConstr(dict_Hstart[Y,t]<=dict_H[Y,t], name = f"Le nombre de centrale {Y} en fonctionnement supérieur au nombre démarré à {t}")
        if t==0:
            model.addConstr(dict_Hstart[Y,t] <= dict_Hstart[Y,t] + dict_H[Y,23], name = f"Nombre max de centrale {X} démarrable à {t}")
        else:
            model.addConstr(dict_Hstart[Y,t] <= dict_Hstart[Y,t] + dict_H[Y,t-1], name = f"Nombre max de centrale {X} démarrable à {t}")
    model.addConstr(gp.quicksum([dict_P[X,t] for X in dict_Thermique])+gp.quicksum(dict_H[Y,t]*dict_Hydro[Y].P for Y in dict_Hydro)==consommation[t], name = f"Equilibre offre-demande à l'instant {t}")
    model.addConstr(gp.quicksum([dict_N[X,t]*dict_Thermique[X].Pmax for X in dict_Thermique]) + gp.quicksum([(1 - dict_H[Y,t]) * dict_Hydro[Y].P for Y in dict_Hydro])>= 1.15 * consommation[t], name = f"Réserve de puissance à {t}")
    # model.addConstr(gp.quicksum([dict_P[X,t] for X in dict_11])==consommation[t])
model.optimize()
        


Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 504 rows, 312 columns and 1176 nonzeros
Model fingerprint: 0x5c72d7cc
Variable types: 72 continuous, 240 integer (96 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e+00, 2e+03]
  Bounds range     [1e+00, 1e+01]
  RHS range        [5e+00, 4e+04]
Found heuristic solution: objective 1381705.0000
Presolve removed 240 rows and 48 columns
Presolve time: 0.00s
Presolved: 264 rows, 264 columns, 744 nonzeros
Variable types: 72 continuous, 192 integer (48 binary)
Found heuristic solution: objective 1359505.0000

Root relaxation: objective 8.886975e+05, 90 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 888697.500    0   30 1359505.00 888697.500  34.

In [37]:
def df_results5(dict_N,dict_P,dict_Nstart,dict_Thermique,dict_Hydro):
    df = pd.DataFrame()
    df["h"] = range(24) 
    df["Consommation (MW)"] = consommation
    df["Production total"] = 0
    df["Coût total"] = 0
    for X in dict_Thermique:
        df[f"Nb centrale {X}"] = [int(dict_N[X,t].X) for t in range(24)]
        df[f"Puissance tot {X}"] = [dict_P[X,t].X for t in range(24)]
        df[f"Coût {X}"] = [dict_N[X,t].X * dict_Thermique[X].Cbase + (dict_P[X,t].X - dict_N[X,t].X * dict_Thermique[X].Pmin) * dict_Thermique[X].Cmwh + dict_Nstart[X,t].X*dict_2[X].Cstart for t in range(24)]
        df["Production total"] += df[f"Puissance tot {X}"]
        df["Coût total"] += df[f"Coût {X}"]
    for Y in dict_Hydro:
        df[f"Fonctionnement {Y}"] = [dict_H[Y,t].X for t in range(24)]
        df[f"Puissance tot {Y}"] = [dict_H[Y,t].X * dict_Hydro[Y].P for t in range(24)]
        df[f"Coût {Y}"] = [dict_H[Y,t].X * dict_Hydro[Y].Cheure + dict_Hstart[Y,t].X * dict_Hydro[Y].Cstart for t in range(24)]
        df["Production total"] += df[f"Puissance tot {Y}"]
        df["Coût total"] += df[f"Coût {Y}"]
    df["Coût MWh"] = df["Coût total"]/df["Production total"]
    return df
df = df_results5(dict_N=dict_N,dict_P=dict_P,dict_Nstart=dict_Nstart,dict_Thermique=dict_Thermique,dict_Hydro=dict_Hydro)
def Results_viz5(df):
    fig = go.Figure()

    Ybis = df["Consommation (MW)"].copy()*0
    for X in dict_Thermique:
        Ybis += df[f"Puissance tot {X}"]
        fig.add_trace(
            go.Scatter(
            x=df["h"], 
            y=Ybis, 
            fill='tonexty',
            name = f"Thermique {X}"
            )
        )
    for Y in dict_Hydro:
        Ybis += df[f"Puissance tot {Y}"]
        fig.add_trace(
            go.Scatter(
            x=df["h"], 
            y=Ybis, 
            fill='tonexty',
            name = f"Hydraulique {Y}"
            )
        )

    # fig.add_trace(
    #     go.Scatter(
    #         x = df["h"],
    #         y = df["Consommation (MW)"],
    #         name = "Consommation"
    #     )
    # )
    fig.add_trace(
        go.Scatter(
            x = df["h"],
            y = df["Production total"],
            name="Production total",
            line = dict(dash='dash',color="red")
        )
    )
    fig.add_trace(
        go.Scatter(
            x=df["h"],
            y=df["Coût MWh"],
            yaxis="y2",
            name="Coût MWh",
            line_color="black"
        )
    )

    fig.update_layout(
        hovermode='x',
        yaxis=dict(title="MW" ,range=[0,df["Consommation (MW)"].max()*1.1]),
        yaxis2=dict(title="€/MWh",
        range=[df["Coût MWh"].min()*0.95,df["Coût MWh"].max()*1.1],
        anchor="free",
        overlaying="y",
        side="right",
        position=1
        ),
        title = "Répartition de la production électrique dans la journée"
    )
    return fig
fig = Results_viz5(df)
fig.write_html("test.html")
fig.show()

### 5.2

On ajoute les variables de décision suivante :
$$
S_t =  \text{Puissance appelée par le pompage à l'instant }t
$$
Puis la contrainte :
$$
S_t d^{(S)} = \sum_Y H_t^{(Y)}d^{(Y)}
$$
Où $d^{(S)}$ représente la hauteur d'eau élevée par MWh et $d^{(Y)}$ la hauteur d'eau prélevée lorsque la centrale $Y$ fonctionne.<br>
On met à jour les contraintes suivante:
$$
\sum_X P_t^{(X)} + \sum_Y H_t^{(Y)}P^{(Y)}-S_t= D_t \text{  ,  Contrainte équilibre offre-demande}
$$

In [38]:
model = gp.Model(name="5.2")

dict_H = {}
dict_Hstart = {}

for Y in dict_Hydro:
    for t in range(24):
        dict_H[Y,t] = model.addVar(vtype=gp.GRB.BINARY, name = f"Centrale Hydro {Y} fonctionne à l'heure {t}")
        dict_Hstart[Y,t] = model.addVar(vtype=gp.GRB.BINARY, name = f"Centrale Hydro {Y} démarre à l'heure {t}")

dict_N = {}
dict_P = {}
dict_Nstart = {}

for t in range(24):
    for X in dict_Thermique:
        dict_N[X,t] = model.addVar(lb=0,ub=dict_Thermique[X].N,vtype=gp.GRB.INTEGER,name=f"Nombre de centrale {X} allumées à {t}h")
        dict_Nstart[X,t] = model.addVar(lb=0,ub=dict_Thermique[X].N,vtype=gp.GRB.INTEGER,name=f"Nombre de centrale {X} démarrées à {t}h")
    for X in dict_Thermique:
        dict_P[X,t] = model.addVar(name=f"Puissance totale {X} à {t}h",vtype=gp.GRB.CONTINUOUS)

dict_S = {t : model.addVar(lb=0,vtype=gp.GRB.CONTINUOUS,name=f"pompage à l'instant {t}") for t in range(24)}
debit_S = 1/3000 # débit de pompage par MWh ou par MW pendant une heure

model.setObjective(
    gp.quicksum([dict_N[element[0],element[1]] * (dict_Thermique[element[0]].Cbase - dict_Thermique[element[0]].Pmin*dict_Thermique[element[0]].Cmwh) for element in product(dict_Thermique,range(24))]) + 
    gp.quicksum([dict_P[element[0],element[1]] * dict_Thermique[element[0]].Cmwh for element in product(dict_Thermique,range(24))])+ 
    gp.quicksum([dict_Nstart[element[0],element[1]]*dict_Thermique[element[0]].Cstart for element in product(dict_Thermique,range(24))])+
    gp.quicksum([dict_H[element[0],element[1]] * dict_Hydro[element[0]].Cheure for element in product(dict_Hydro,range(24))])+
    gp.quicksum([dict_Hstart[element[0],element[1]] * dict_Hydro[element[0]].Cstart for element in product(dict_Hydro,range(24))]),
    gp.GRB.MINIMIZE
    )

for t in range(24):
    for X in dict_Thermique:
        model.addConstr(dict_P[X,t]<=dict_N[X,t] * dict_Thermique[X].Pmax, name=f"borne sup puissance, {X} à {t}h")
        model.addConstr(dict_P[X,t]>=dict_N[X,t] * dict_Thermique[X].Pmin, name=f"borne inf puissance, {X} à {t}h")
        model.addConstr(dict_Nstart[X,t]<=dict_N[X,t], name = f"Le nombre de centrale {X} en fonctionnement supérieur au nombre démarré à {t}")
        if t ==0:
            model.addConstr(dict_Nstart[X,t]<= dict_Thermique[X].N - dict_N[X,23], name = f"Nombre max de centrale {X} démarable à {t}")
            model.addConstr(dict_N[X,t] <= dict_Nstart[X,t] + dict_N[X,23], name = f"Nombre max de centrale {X} en fonctionnement à {t}")
        else:
            model.addConstr(dict_Nstart[X,t]<= dict_Thermique[X].N - dict_N[X,t-1], name = f"Nombre max de centrale {X} démarable à {t}")
            model.addConstr(dict_N[X,t] <= dict_Nstart[X,t] + dict_N[X,t-1], name = f"Nombre max de centrale {X} en fonctionnement à {t}")
    for Y in dict_Hydro:
        model.addConstr(dict_Hstart[Y,t]<=dict_H[Y,t], name = f"Le nombre de centrale {Y} en fonctionnement supérieur au nombre démarré à {t}")
        if t==0:
            model.addConstr(dict_Hstart[Y,t] <= dict_Hstart[Y,t] + dict_H[Y,23], name = f"Nombre max de centrale {X} démarrable à {t}")
        else:
            model.addConstr(dict_Hstart[Y,t] <= dict_Hstart[Y,t] + dict_H[Y,t-1], name = f"Nombre max de centrale {X} démarrable à {t}")
    model.addConstr(
        gp.quicksum([dict_P[X,t] for X in dict_Thermique])+gp.quicksum(dict_H[Y,t]*dict_Hydro[Y].P for Y in dict_Hydro) - dict_S[t] ==consommation[t], 
        name = f"Equilibre offre-demande à l'instant {t}")
    model.addConstr(gp.quicksum([dict_N[X,t]*dict_Thermique[X].Pmax for X in dict_Thermique]) + gp.quicksum([(1 - dict_H[Y,t]) * dict_Hydro[Y].P for Y in dict_Hydro])>= 1.15 * consommation[t], name = f"Réserve de puissance à {t}")
    # model.addConstr(gp.quicksum([dict_P[X,t] for X in dict_11])==consommation[t])
model.addConstr(
    gp.quicksum([dict_S[t] * debit_S - gp.quicksum([dict_H[Y,t] * dict_Hydro[Y].debit for Y in dict_Hydro]) for t in range(24)])==0,
    name="Equilibre niveau réservoir"
)
model.optimize()
        


Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 505 rows, 336 columns and 1272 nonzeros
Model fingerprint: 0x381e6735
Variable types: 96 continuous, 240 integer (96 binary)
Coefficient statistics:
  Matrix range     [3e-04, 4e+03]
  Objective range  [1e+00, 2e+03]
  Bounds range     [1e+00, 1e+01]
  RHS range        [5e+00, 4e+04]
Presolve removed 240 rows and 48 columns
Presolve time: 0.00s
Presolved: 265 rows, 288 columns, 840 nonzeros
Variable types: 96 continuous, 192 integer (48 binary)
Found heuristic solution: objective 1152895.0000
Found heuristic solution: objective 1026475.0000

Root relaxation: objective 9.837414e+05, 130 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 983741.429    0   30 1026475.00 983741.429  4.

In [42]:
def df_results52(dict_N,dict_P,dict_Nstart,dict_Thermique,dict_Hydro,dict_S):
    df = pd.DataFrame()
    df["h"] = range(24) 
    df["Consommation (MW)"] = consommation
    df["Production total"] = 0
    df["Coût total"] = 0
    for X in dict_Thermique:
        df[f"Nb centrale {X}"] = [int(dict_N[X,t].X) for t in range(24)]
        df[f"Puissance tot {X}"] = [dict_P[X,t].X for t in range(24)]
        df[f"Coût {X}"] = [dict_N[X,t].X * dict_Thermique[X].Cbase + (dict_P[X,t].X - dict_N[X,t].X * dict_Thermique[X].Pmin) * dict_Thermique[X].Cmwh + dict_Nstart[X,t].X*dict_2[X].Cstart for t in range(24)]
        df["Production total"] += df[f"Puissance tot {X}"]
        df["Coût total"] += df[f"Coût {X}"]
    for Y in dict_Hydro:
        df[f"Fonctionnement {Y}"] = [dict_H[Y,t].X for t in range(24)]
        df[f"Démarrage {Y}"] = [dict_Hstart[Y,t].X for t in range(24)]
        df[f"Puissance tot {Y}"] = [dict_H[Y,t].X * dict_Hydro[Y].P for t in range(24)]
        df[f"Coût {Y}"] = [dict_H[Y,t].X * dict_Hydro[Y].Cheure + dict_Hstart[Y,t].X * dict_Hydro[Y].Cstart for t in range(24)]
        df["Production total"] += df[f"Puissance tot {Y}"]
        df["Coût total"] += df[f"Coût {Y}"]
    df["Pompage"] = [-dict_S[t].X for t in range(24)]
    df["Production total"] += df["Pompage"]
    df["Réservoir variation"] = [dict_S[t].X * debit_S - sum([dict_H[Y,t].X * dict_Hydro[Y].debit for Y in dict_Hydro]) for t in range(24)]
    df["Réservoir"] = df["Réservoir variation"].cumsum()
    df["Coût MWh"] = df["Coût total"]/df["Production total"]
    return df
df = df_results52(dict_N=dict_N,dict_P=dict_P,dict_Nstart=dict_Nstart,dict_Thermique=dict_Thermique,dict_Hydro=dict_Hydro,dict_S=dict_S)

def Results_viz52(df):
    fig = go.Figure()

    Ybis = df["Consommation (MW)"].copy()*0
    for X in dict_Thermique:
        Ybis += df[f"Puissance tot {X}"]
        fig.add_trace(
            go.Scatter(
            x=df["h"], 
            y=Ybis, 
            fill='tonexty',
            name = f"Thermique {X}"
            )
        )
    for Y in dict_Hydro:
        Ybis += df[f"Puissance tot {Y}"]
        fig.add_trace(
            go.Scatter(
            x=df["h"], 
            y=Ybis, 
            fill='tonexty',
            name = f"Hydraulique {Y}"
            )
        )
    fig.add_trace(
        go.Scatter(
        x=df["h"], 
        y=df["Pompage"], 
        fill='tozeroy',
        name = "Pompage"
        )
    )
    
    # fig.add_trace(
    #     go.Scatter(
    #         x = df["h"],
    #         y = df["Consommation (MW)"],
    #         name = "Consommation"
    #     )
    # )
    fig.add_trace(
        go.Scatter(
            x = df["h"],
            y = df["Production total"],
            name="Production total",
            line = dict(dash='dash',color="red")
        )
    )
    fig.add_trace(
        go.Scatter(
            x=df["h"],
            y=df["Coût MWh"],
            yaxis="y2",
            name="Coût MWh",
            line_color="black"
        )
    )
    fig.add_trace(
        go.Scatter(
            x=df["h"],
            y=df["Réservoir"],
            yaxis="y3",
            name="Niveau réservoir",
            line_color="grey"
        )
    )

    fig.update_layout(
        hovermode='x',
        yaxis=dict(title="MW" ,range=[df["Pompage"].min()*1.1,df["Consommation (MW)"].max()*1.1]),
        yaxis2=dict(
            title="€/MWh",
            range=[df["Coût MWh"].min()*0.95,df["Coût MWh"].max()*1.1],
            anchor="free",
            overlaying="y",
            side="right",
            position=1
            ),
        yaxis3=dict(
            title="Réservoir (m)",
            range=[df["Réservoir"].min()*1.1,df["Réservoir"].max()*1.1],
            anchor="free",
            overlaying="y",
            side="left",
            position=0.05
            ),
        title = "Répartition de la production électrique dans la journée"
    )
    return fig
fig = Results_viz52(df)
fig.write_html("test.html")
fig.show()

In [43]:
df

,h,Consommation (MW),Production total,Coût total,Nb centrale A,Puissance tot A,Coût A,Nb centrale B,Puissance tot B,Coût B,...,Puissance tot 9,Coût 9,Fonctionnement 14,Démarrage 14,Puissance tot 14,Coût 14,Pompage,Réservoir variation,Réservoir,Coût MWh
0,0,15000,15000.0,21750.0,12,10200.0,12000.0,3,5250.0,9750.0,...,-0.0,0.0,-0.0,0.0,-0.0,0.0,-450.0,0.150000,1.500000e-01,1.450000
1,1,15000,15000.0,21750.0,12,10200.0,12000.0,3,5250.0,9750.0,...,-0.0,0.0,-0.0,0.0,-0.0,0.0,-450.0,0.150000,3.000000e-01,1.450000
2,2,15000,15000.0,21750.0,12,10200.0,12000.0,3,5250.0,9750.0,...,-0.0,0.0,-0.0,0.0,-0.0,0.0,-450.0,0.150000,4.500000e-01,1.450000
3,3,15000,15000.0,21750.0,12,10200.0,12000.0,3,5250.0,9750.0,...,-0.0,0.0,-0.0,0.0,-0.0,0.0,-450.0,0.150000,6.000000e-01,1.450000
4,4,15000,15000.0,21750.0,12,10200.0,12000.0,3,5250.0,9750.0,...,-0.0,0.0,-0.0,0.0,-0.0,0.0,-450.0,0.150000,7.500000e-01,1.450000
5,5,15000,15000.0,21750.0,12,10200.0,12000.0,3,5250.0,9750.0,...,-0.0,0.0,-0.0,0.0,-0.0,0.0,-450.0,0.150000,9.000000e-01,1.450000
6,6,30000,30000.0,50990.0,12,11950.0,15500.0,9,15750.0,35250.0,...,900.0,90.0,1.0,0.0,1400.0,150.0,-0.0,-0.780000,1.200000e-01,1.699667
7,7,30000,30000.0,44990.0,12,11950.0,15500.0,9,15750.0,29250.0,...,900.0,90.0,1.0,0.0,1400.0,150.0,-0.0,-0.780000,-6.600000e-01,1.499667
8,8,30000,30000.0,45810.0,12,12360.0,16320.0,9,15750.0,29250.0,...,900.0,90.0,1.0,0.0,1400.0,150.0,-410.0,-0.643333,-1.303333e+00,1.527000
9,9,25000,25000.0,41250.0,12,10200.0,12000.0,9,15750.0,29250.0,...,-0.0,0.0,-0.0,0.0,-0.0,0.0,-950.0,0.316667,-9.866667e-01,1.650000


### 5.3 Paliers de fonctionnement

Les variables de décisions concernant l'hydroélectricité deviennent :
$$
H_t^{(Y,n)} \in \{0,1\} = \text{La centrale hydroélectrique } Y \text{ fonctionne au palier } n \text{ à l'instant } t
$$
$$
H_{start,t}^{(Y)} \in \{0,1\} = \text{  La centrale hydroélectrique $Y$ démarre à l'instant $t$}
$$
Les contraintes suivantes sont modifiées :

$$
\text{Equilibre offre-demande : } \sum_X P_t^{(X)} + \sum_{Y,n} H_t^{(Y,n)}P^{(Y,n)}-S_t= D_t
$$
$$
\text{Réserve de puissance : } \sum_X N_t^{(X)}P_{max}^{(X)} + \sum_Y P^{(Y,n_{max})}\geq D_t\times 1,15
$$
$$
\text{Niveau réservoir : } S_t d^{(S)} = \sum_{Y,n} H_t^{(Y,n)}d^{(Y,n)}
$$

Les contraintes sur les démarrages et les paliers deviennent :
$$
\text{Un seul palier fonctionne à la fois , }\sum_n H_t^{(Y,n)} \leq 1
$$
$$
\text{ Si un palier fonctionne alors il y avait déjà un palier actif ou la centrale est démarrée , } H_t^{(Y,n)} \leq \sum_n H_{t-1}^{(Y,n)} + H_{start,t}^{(Y)}
$$
Modification de la fonction objectif :
$$
\text{Minimiser} \sum_X \sum_t (P_t^{(X)} - P_{min}^{(X)}N_t^{(X)}) C_{MWh}^{(X)} + N_t^{(X)}C_{base}^{(X)} + N_{start,t}^{(X)}C_{start}^{(X)} + \sum_{Y,n} \sum_t H_t^{(Y,n)}C_{base}^{(Y,n)} + \sum_Y H_{start,t}^{(Y)}C_{start}^{(Y)}
$$
Où $P^{(Y,n)}$ est la puissance de la centrale $Y$ au palier $n$, $d^{(Y,n)}$ est l'abaissement sur une heure quand la centrale $Y$ fonctionne au palier $n$ et $C_{base}^{(Y,n)}$ est le coût du fonctionnement de la centrale $Y$ pendant une heure de fonctionnement au palier $n$

### 5.4 Exclusion pompage et génération hydro

On introduit la variable de décision suivante :
$$
N^{S}_t \in \{0,1\} = \text{La pompage est en fonctionnement}
$$
On ajoute les contraintes suivante :
$$
\text{Si une centrale hydroélectrique est activée alors le pompage est désactivé , } \frac{1}{\text{Nb centrale hydro}}\sum_{Y,n}H_t^{Y,n} \leq 1 - N_t^{(S)}
$$
$$
\text{Si aucune centrale hydroélectrique est allumée alors il peut y avoir du pompage , } S_t \leq M \times ( \sum_{Y,n} H_t^{(Y,n)})
$$